<a href="https://colab.research.google.com/github/kbhavana14/machine_learning_pyton_fcc/blob/main/Tensorflow_FCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn

  Preparing metadata (setup.py) ... done


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


In [3]:
dftrain = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dfeval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
#dftrain.head()
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [4]:
categorical_data = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
numeric_data = ['age','fare']

feature_columns=[]
for feature_name in categorical_data:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in numeric_data:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))

feature_columns


[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.strin

TRAINING

In [1]:
# feed the input
# batches - 32
# epochs - no of time our model sees the dataset
# input function is used to feed the input data in batches

Input Function

In [8]:
# converts pandas data frame into the tf.data.Dataset object(the on=bj that the model use)

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000) # randomize order of data
        ds = ds.batch(batch_size).repeat(num_epochs) # split the dataset into batches of 32 and repeat for no of epochs times
        return ds # return batch for data set
    return input_function # return a function obj for use

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

Creating the model

In [9]:
# we use a linear estimator for linear reg algo
# we create a linear estimator by passing feature columns that we created
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

Training the model

In [12]:
linear_est.train(train_input_fn) # train
result = linear_est.evaluate(eval_input_fn) # get model stats by testing on testing data

clear_output()
print(result)
print(result['accuracy']) # res has the dict of stats of the model

{'accuracy': 0.7765151, 'accuracy_baseline': 0.625, 'auc': 0.8358127, 'auc_precision_recall': 0.7980027, 'average_loss': 0.47021803, 'label/mean': 0.375, 'loss': 0.45702145, 'precision': 0.6960784, 'prediction/mean': 0.40272665, 'recall': 0.7171717, 'global_step': 600}
0.7765151


In [23]:
# actual predictions
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[4])
print(y_eval.loc[4])
print(result[4]['probabilities'][1]) # 1 is chance of survival

sex                          male
age                          34.0
n_siblings_spouses              0
parch                           0
fare                         13.0
class                      Second
deck                            D
embark_town           Southampton
alone                           y
Name: 4, dtype: object
1
0.29968792
